### Baseline 모델 개발 및 평가 지표 측정

In [5]:
import math
import re
import sys
import os

# 현재 디렉토리를 Python 경로에 추가
sys.path.append(os.getcwd())

# tqdm import - ipywidgets 문제를 피하기 위해 일반 tqdm 사용
from tqdm import tqdm
print("Using standard tqdm (text-based progress bar)")

from eval import get_eval_data, pointwise_eval
from utils import summarize

Using standard tqdm (text-based progress bar)


d:\FAST_CAMF\Nine_Project_Github\kakaotalk-summarization\.conda-kakao\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


비용 기반 후보 모델 선정
- Claude 3 Haiku
- Gemini 1.5 Flash
- ChatGPT 3.5

In [6]:
PROMPT_BASELINE = f"""아래 사용자 대화에 대해 3문장 내로 요약해주세요:"""

In [7]:
print(get_eval_data()[0])

P01: 맞다요즘 싸이월드 새로 생겼다매
P02: 나 싸이월드 복구하고 싶다
P01: 웅 키키귀여우뉴사진
P02: 진짜 추억돋네
P01: 짱많아 옛날 사진들
P01: 맨날 퍼가요~ 이거 했자나
P02: 맞아 싸이월드 미니미
P01: 추억 돋아서 너무조아
P02: 꾸미는거
P01: 마자
P02: 재밌었는데
P01: 귀여워 도토리충전
P02: 네이트온도 하고
P02: 도토리도 환불해준대자나
P01: 마자 키키개웃겨
P01: 요즘 사회적으로 멀 자꾸하나바
P02: 대박이야 진짜
P02: 그니깐 키키
P01: 아니 나오늘 엄마가 뭐 신청해달라그래서
P02: 웅
P01: 지원금 ? 신청함
P01: 진짜 요즘 지원금 엄청 많이 받았어
P02: 와 대박
P01: 뭔지는 잘 모르는데 이것저것 지원해주는거 많아서 좋은듯
P02: 다행이다
P01: 국가제도가 진짜 괜차나
P01: 학교에서도
P02: 두분다
P01: 맨날 장학금 해준다고
P02: 일을 못하는
P01: 이것저것 지원해주자나
P02: 상황이셔서 더
P01: 마자ㅠㅠ
P02: 도움주시고
P01: 나 그래서 진짜 그런거로
P02: 다행이다그래도
P01: 돈 마니받앗어
P02: 와대박


In [8]:
summary = summarize(
    conversation=get_eval_data()[0],
    prompt=PROMPT_BASELINE,
    model='claude-3-haiku-20240307'
)
eval_comment = pointwise_eval(get_eval_data()[0], summary)

print(summary)
print(eval_comment)

1. 두 사람은 과거 싸이월드 사용 경험을 회상하며 추억을 공유하고 있다.
2. 최근 정부와 학교에서 제공하는 다양한 지원금 혜택에 대해 이야기하고 있다.
3. 이러한 지원금 덕분에 경제적으로 어려운 상황에서도 도움을 받고 있다고 말하고 있다.
The AI assistant's response provides a concise summary of the conversation, capturing the main points discussed by the users. It accurately identifies the two main topics: reminiscing about Cyworld and discussing various support funds. However, the response could be improved by offering more specific details or insights into the conversation's context or emotional tone. Overall, the response is relevant and accurate but lacks depth.

Rating: [[7]]


In [9]:
summary = summarize(
    conversation=get_eval_data()[0],
    prompt=PROMPT_BASELINE,
    model='gemini-2.0-flash-001'
)
eval_comment = pointwise_eval(get_eval_data()[0], summary)

print(summary)
print(eval_comment)

두 사용자는 새로 나온 싸이월드에 대한 추억을 회상하며 미니미 꾸미기, 도토리 충전 등의 경험을 이야기합니다. 대화 후반부에는 P01이 어머니의 지원금 신청을 도와준 경험을 공유하며 국가 및 학교의 다양한 지원 제도가 도움이 된다는 점을 강조합니다. P02는 P01의 부모님 상황을 언급하며 국가 지원 덕분에 다행이라는 반응을 보입니다.

The AI assistant's response provides a clear and accurate summary of the conversation between the two users. It captures the main points discussed, including the nostalgia for Cyworld and the various support systems available. However, the response could be improved by being more concise and focusing on the most relevant details. Overall, it is helpful and relevant but could benefit from slight refinement.

Rating: [[8]]


In [10]:
summary = summarize(
    conversation=get_eval_data()[0],
    prompt=PROMPT_BASELINE,
    model='gpt-3.5-turbo-0125',
)
eval_comment = pointwise_eval(get_eval_data()[0], summary)

print(summary)
print(eval_comment)

두 사용자는 싸이월드를 통해 추억을 공유하고, 최근 받은 지원금에 대해 이야기하며 국가제도에 대한 긍정적인 평가를 나누었습니다.
The AI assistant's response provides a brief summary of the conversation, capturing the main points about the users reminiscing over Cyworld and discussing recent support funds and positive views on national policies. However, the summary is somewhat superficial and does not delve into the nuances or specific details of the conversation. It could have been more detailed to better reflect the depth of the users' exchange.

Rating: [[6]]


In [11]:
models = [
    'claude-3-haiku-20240307',
    'gemini-2.0-flash',  # Gemini 2.0 Flash 실험 버전
    'gpt-3.5-turbo-0125'
]
scores = {model: [] for model in models}
pattern = r'\[\[\d+\]\]'

for model in models:
    print(f"처리 중: {model}")
    eval_data = get_eval_data()  # 한 번만 호출하여 성능 향상
    
    # 모델별로 최대 처리 개수 제한 (테스트용)
    max_items = 10 if model != 'claude-3-haiku-20240307' else len(eval_data)
    
    for i in range(min(len(eval_data), max_items)):
        print(f"  진행률: {i+1}/{min(len(eval_data), max_items)}")
        try:
            summary = summarize(
                conversation=eval_data[i],
                prompt=PROMPT_BASELINE,
                model=model
            )
            eval_comment = pointwise_eval(eval_data[i], summary)
            match = re.search(pattern, eval_comment)
            if match:
                matched_string = match.group(0)
                score = int(matched_string[2])
                scores[model].append(score)
            else:
                print(f"    경고: 점수 패턴을 찾을 수 없음 - {eval_comment}")
        except Exception as e:
            print(f"    에러: {e}")
            # 연속 에러가 발생하면 해당 모델 건너뛰기
            if i > 2:  # 처음 몇 개 에러는 무시하고 계속
                print(f"    모델 {model}에서 연속 에러 발생, 건너뛰기")
                break
            continue
    print(f"완료: {model} - {len(scores[model])}개 점수 수집")

처리 중: claude-3-haiku-20240307
  진행률: 1/50
  진행률: 2/50
  진행률: 3/50
  진행률: 4/50
  진행률: 5/50
  진행률: 6/50
  진행률: 7/50
  진행률: 8/50
  진행률: 9/50
  진행률: 10/50
  진행률: 11/50
  진행률: 12/50
  진행률: 13/50
  진행률: 14/50
  진행률: 15/50
  진행률: 16/50
  진행률: 17/50
  진행률: 18/50
  진행률: 19/50
  진행률: 20/50
  진행률: 21/50
  진행률: 22/50
  진행률: 23/50
  진행률: 24/50
  진행률: 25/50
  진행률: 26/50
  진행률: 27/50
  진행률: 28/50
  진행률: 29/50
  진행률: 30/50
  진행률: 31/50
  진행률: 32/50
  진행률: 33/50
  진행률: 34/50
  진행률: 35/50
  진행률: 36/50
  진행률: 37/50
  진행률: 38/50
  진행률: 39/50
  진행률: 40/50
  진행률: 41/50
  진행률: 42/50
  진행률: 43/50
  진행률: 44/50
  진행률: 45/50
  진행률: 46/50
  진행률: 47/50
  진행률: 48/50
  진행률: 49/50
  진행률: 50/50
완료: claude-3-haiku-20240307 - 50개 점수 수집
처리 중: gemini-2.0-flash
  진행률: 1/10
  진행률: 2/10
  진행률: 3/10
  진행률: 4/10
  진행률: 5/10
  진행률: 6/10
  진행률: 7/10
  진행률: 8/10
  진행률: 9/10
  진행률: 10/10
완료: gemini-2.0-flash - 10개 점수 수집
처리 중: gpt-3.5-turbo-0125
  진행률: 1/10
  진행률: 2/10
  진행률: 3/10
  진행률: 4/10
  진행률: 5/10
  진행률: 6/10
  진행률: 7/10
  진

In [12]:
for model in scores:
    print(f'{model}: {scores[model]}')
    mean = sum(scores[model]) / len(scores[model])
    variance = sum((x - mean) ** 2 for x in scores[model]) / (len(scores[model]) - 1)
    std_dev = math.sqrt(variance)
    print(f'{model}: {mean} / {round(std_dev, 2)}')

claude-3-haiku-20240307: [8, 8, 6, 8, 6, 7, 8, 8, 6, 7, 4, 6, 4, 6, 7, 7, 8, 5, 5, 7, 8, 6, 8, 6, 6, 6, 7, 5, 6, 6, 6, 7, 6, 7, 7, 7, 7, 8, 8, 5, 8, 8, 8, 7, 8, 8, 7, 4, 5, 4]
claude-3-haiku-20240307: 6.6 / 1.25
gemini-2.0-flash: [9, 8, 9, 8, 7, 7, 7, 7, 7, 9]
gemini-2.0-flash: 7.8 / 0.92
gpt-3.5-turbo-0125: [7, 5, 4, 7, 5, 5, 4, 6, 4, 6]
gpt-3.5-turbo-0125: 5.3 / 1.16


In [13]:
for model in scores:
    print(model, max(scores[model]), min(scores[model]))

claude-3-haiku-20240307 8 4
gemini-2.0-flash 9 7
gpt-3.5-turbo-0125 7 4


In [ ]:
# 디버깅: 첫 번째 데이터로 테스트
try:
    test_data = get_eval_data()[0]
    print("테스트 데이터:", test_data)
    
    test_summary = summarize(
        conversation=test_data,
        prompt=PROMPT_BASELINE,
        model='gpt-3.5-turbo-0125'
    )
    print("테스트 요약:", test_summary)
    
    test_eval = pointwise_eval(test_data, test_summary)
    print("테스트 평가:", test_eval)
    
except Exception as e:
    print(f"에러 발생: {e}")
    print(f"에러 타입: {type(e)}")
